# Importación de librerías

In [2]:
import numpy as np
import cv2
import os
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from skimage import img_as_ubyte
import matplotlib.pyplot as plt

# Procesamiento de imagenes awake (Duración: ~5 minutos)

In [10]:
# Rutas de imágenes
ruta = "../images/awake/"
ruta_procesadas_awake = os.path.join(ruta, "../../processed_images/awake")
os.makedirs(ruta_procesadas_awake, exist_ok=True)

for nombre in sorted(os.listdir(ruta)):
    ruta_img = os.path.join(ruta, nombre)
    # Saltamos si no es archivo de imagen (.jpg, .png, etc.)
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue

    img = cv2.imread(ruta_img)
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gris = cv2.GaussianBlur(gris, (5, 5), 1)
    _, binaria = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
    kernel = np.ones((3, 3), np.uint8)
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_OPEN, kernel, iterations=1)
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_CLOSE, kernel, iterations=1)
    binaria = cv2.resize(binaria, (1024, 1024))
    # Guardar imagen procesada
    ruta_destino = os.path.join(ruta_procesadas_awake, nombre)
    cv2.imwrite(ruta_destino, binaria)

# Procesamiento de imagenes sleepy (Duración: ~5 minutos)

In [9]:
# Rutas de imágenes
ruta = "../images/sleepy/"
ruta_procesadas_sleepy = os.path.join(ruta, "../../processed_images/sleepy")
os.makedirs(ruta_procesadas_sleepy, exist_ok=True)

for nombre in sorted(os.listdir(ruta)):
    ruta_img = os.path.join(ruta, nombre)
    # Saltamos si no es archivo de imagen (.jpg, .png, etc.)
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue

    img = cv2.imread(ruta_img)
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gris = cv2.GaussianBlur(gris, (5, 5), 1)
    _, binaria = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
    kernel = np.ones((3, 3), np.uint8)
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_OPEN, kernel, iterations=1)
    binaria = cv2.morphologyEx(binaria, cv2.MORPH_CLOSE, kernel, iterations=1)
    binaria = cv2.resize(binaria, (1024, 1024))
    # Guardar imagen procesada
    ruta_destino = os.path.join(ruta_procesadas_sleepy, nombre)
    cv2.imwrite(ruta_destino, binaria)

# Extraer estadisticas basicas para awake y sleepy (Duración: ~18 minutos)

In [ ]:
ruta = "../images/awake/"
ruta_procesadas_sleepy = os.path.join(ruta, "../../processed_images/sleepy")
ruta_procesadas_awake = os.path.join(ruta, "../../processed_images/awake")
resultados = []

for nombre in sorted(os.listdir(ruta_procesadas_awake)):
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue

    ruta_img = os.path.join(ruta_procesadas_awake, nombre)
    image = cv2.imread(ruta_img, cv2.IMREAD_GRAYSCALE)

    mean = np.mean(image)
    variance = np.var(image)
    std_dev = np.std(image)
    image_float = np.float32(image) + 1e-5
    entropy = -np.sum(image_float * np.log(image_float))

    resultados.append({
        'image': nombre,
        'mean': mean,
        'variance': variance,
        'std_dev': std_dev,
        'entropy': entropy
    })

# Guardar en hoja de Excel (crea el archivo si no existe, sino sobrescribe la hoja)
excel_path = "../results/stats.xlsx"
df = pd.DataFrame(resultados)
with pd.ExcelWriter(excel_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="basics_stats_awake", index=False)

print("Estadísticas básicas guardadas en la hoja 'basics_stats_awake' de 'stats.xlsx'")


resultados2 = []

for nombre in sorted(os.listdir(ruta_procesadas_sleepy)):
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue

    ruta_img = os.path.join(ruta_procesadas_sleepy, nombre)
    image = cv2.imread(ruta_img, cv2.IMREAD_GRAYSCALE)

    mean = np.mean(image)
    variance = np.var(image)
    std_dev = np.std(image)
    image_float = np.float32(image) + 1e-5
    entropy = -np.sum(image_float * np.log(image_float))

    resultados2.append({
        'image': nombre,
        'mean': mean,
        'variance': variance,
        'std_dev': std_dev,
        'entropy': entropy
    })

# Guardar en hoja de Excel (crea el archivo si no existe, sino sobrescribe la hoja)
excel_path = "../results/stats.xlsx"
df = pd.DataFrame(resultados2)
with pd.ExcelWriter(excel_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="basics_stats_sleepy", index=False)

print("Estadísticas básicas guardadas en la hoja 'basics_stats_sleepy' de 'stats.xlsx'")

Estadísticas básicas guardadas en la hoja 'basics_stats_sleepy' de 'stats.xlsx'


# Estadisticas de segundo orden para awake y sleepy (Duración: ~10 minutos)

In [ ]:
ruta = "../images/awake/"
ruta_procesadas_sleepy = os.path.join(ruta, "../../processed_images/sleepy")
ruta_procesadas_awake = os.path.join(ruta, "../../processed_images/awake")

# Awake

resultados = []

for nombre in sorted(os.listdir(ruta_procesadas_awake)):
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue
    
    ruta_img = os.path.join(ruta_procesadas_awake, nombre)
    image = cv2.imread(ruta_img, cv2.IMREAD_GRAYSCALE)

    # Convertir a ubyte
    image = img_as_ubyte(image)

    # Calcular GLCM (1 pixel distancia, ángulo 0)
    glcm = graycomatrix(image, distances=[1], angles=[0], symmetric=True, normed=True)

    # Extraer propiedades
    contrast = graycoprops(glcm, prop='contrast')[0, 0]
    homogeneity = graycoprops(glcm, prop='homogeneity')[0, 0]
    dissimilarity = graycoprops(glcm, prop='dissimilarity')[0, 0]
    energy = graycoprops(glcm, prop='energy')[0, 0]
    correlation = graycoprops(glcm, prop='correlation')[0, 0]
    mean_glcm = np.mean(glcm)
    std_dev_glcm = np.std(glcm)
    glcm_flat = glcm.flatten()
    glcm_flat = glcm_flat[glcm_flat > 0]
    entropy_glcm = -np.sum(glcm_flat * np.log(glcm_flat))

    resultados.append({
        'image': nombre,
        'contrast': contrast,
        'homogeneity': homogeneity,
        'dissimilarity': dissimilarity,
        'energy': energy,
        'correlation': correlation,
        'mean_glcm': mean_glcm,
        'std_dev_glcm': std_dev_glcm,
        'entropy_glcm': entropy_glcm
    })

# Crear DataFrame
df_glcm = pd.DataFrame(resultados)

# Escribir en hoja nueva de Excel
excel_path = "../results/stats.xlsx"  # Usar siempre el mismo archivo
with pd.ExcelWriter(excel_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df_glcm.to_excel(writer, sheet_name="GLCM_awake", index=False)

print(f"Características GLCM guardadas en la hoja 'GLCM_awake' de '{excel_path}'")

# Sleepy

resultados2 = []

for nombre in sorted(os.listdir(ruta_procesadas_sleepy)):
    if not nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        continue
    
    ruta_img = os.path.join(ruta_procesadas_sleepy, nombre)
    image = cv2.imread(ruta_img, cv2.IMREAD_GRAYSCALE)

    # Convertir a ubyte
    image = img_as_ubyte(image)

    # Calcular GLCM (1 pixel distancia, ángulo 0)
    glcm = graycomatrix(image, distances=[1], angles=[0], symmetric=True, normed=True)

    # Extraer propiedades
    contrast = graycoprops(glcm, prop='contrast')[0, 0]
    homogeneity = graycoprops(glcm, prop='homogeneity')[0, 0]
    dissimilarity = graycoprops(glcm, prop='dissimilarity')[0, 0]
    energy = graycoprops(glcm, prop='energy')[0, 0]
    correlation = graycoprops(glcm, prop='correlation')[0, 0]
    mean_glcm = np.mean(glcm)
    std_dev_glcm = np.std(glcm)
    glcm_flat = glcm.flatten()
    glcm_flat = glcm_flat[glcm_flat > 0]
    entropy_glcm = -np.sum(glcm_flat * np.log(glcm_flat))

    resultados2.append({
        'image': nombre,
        'contrast': contrast,
        'homogeneity': homogeneity,
        'dissimilarity': dissimilarity,
        'energy': energy,
        'correlation': correlation,
        'mean_glcm': mean_glcm,
        'std_dev_glcm': std_dev_glcm,
        'entropy_glcm': entropy_glcm
    })

# Crear DataFrame
df_glcm = pd.DataFrame(resultados2)

# Escribir en hoja nueva de Excel
excel_path = "../results/stats.xlsx"  # Usar siempre el mismo archivo
with pd.ExcelWriter(excel_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df_glcm.to_excel(writer, sheet_name="GLCM_sleepy", index=False)

print(f"Características GLCM guardadas en la hoja 'GLCM_sleepy' de '{excel_path}'")